Table of contents

* [Out-of-Band Domain Threat Hunt](#out-of-band-domain-threat-hunt)
  * [Hypothesis](#hypothesis)
  * [Investigation](#investigation)
  * [Requirements](#requirements)
* [Analysis](#analysis)
  * [Are attackers using common Out-of-band application security testing (OAST) domains?](#are-attackers-using-common-out-of-band-application-security-testing-oast-domains-–-eg-oastifycom-interactsh-oastfun)
    * [Distribution of OAST domains](#distribution-of-oast-domains)
    * [Breakdown of OAST domains by attack type](#breakdown-of-oast-domains-by-attack-type)
  * [Is the attacker server still live? Can we grab the payload from it? ](#is-the-attacker-server-still-live-can-we-grab-the-payload-from-it)
    * [Sandbox scan of OOB Domain using urlscan.io](#sandbox-scan-of-oob-domain-using-urlscanio)
    * [Display screenshot of scanned url](#display-screenshot-of-scanned-url)

# Out-of-Band Domain Threat Hunt

## Hypothesis
Attackers use techniques that callback to their out-of-band (OOB) domains to exploit vulnerabilities, bypass security controls, and exfiltrate data. These domains may appear in WAF logs. By analyzing WAF logs for unusual domain names not typical of our traffic, we can identify potential threats and respond accordingly.

<img src="https://www.fastly.com/cimages/6pk8mg3yh2ee/2PCCPThjdCOr5NUpWT03hG/751e27d8f2bb38dd0a4bf95ea56fcceb/Figure_7_captionsvg.svg" width="600" height="300"/></img>

## Investigation

The first thing needed for any threat hunt is data. In this case, we are focused on WAF logs are based on alerts of potential malice. As a Fastly Next-Gen WAF (NGWAF) customer, searching through a site’s request feed is an excellent source for this data.

You can search by filtering on tags, in particular the tag object fields type and value. Through the use of the newly introduced Out-of-Band Domain (OOB-DOMAIN) signal we can filter and aggregate based on this signal.

## Requirements

*   [NGWAF API Key](https://docs.fastly.com/en/ngwaf/using-our-api)
*   (optional) [URLScan.io API Key](https://urlscan.io/docs/api/)

Configure these environment variables.

* SIGSCI_CORP
* SIGSCI_EMAIL
* SIGSCI_SITE
* SIGSCI_TOKEN
* URLSCAN_API_KEY

In [ ]:
%pip install python-dotenv

In [ ]:
import requests, os, json
from dotenv import load_dotenv

In [ ]:

# Load environment variables
load_dotenv()

email = os.getenv('SIGSCI_EMAIL')
token = os.getenv('SIGSCI_TOKEN')
corp = os.getenv('SIGSCI_CORP')
site = os.getenv('SIGSCI_SITE')

url = 'https://dashboard.signalsciences.net'

headers = {
    'Content-type': 'application/json',
    'x-api-user': email,
    'x-api-token': token
}

endpoint = f"/api/v0/corps/{corp}/sites/{site}/requests"

In [ ]:
# search for requests tagged with OOB-DOMAIN
params = {"q": "from:-6h tag:OOB-DOMAIN"}
resp = requests.get(url + endpoint, headers=headers, params=params)
search_results = resp.json()['data']

while 'next' in resp.json():
    next_uri = resp.json()['next']['uri']
    resp = requests.get(url + next_uri, headers=headers)
    search_results.extend(resp.json()['data'])

print(f"Found {len(search_results)} results that contain OOB domains")

## Analysis

### Are attackers using common Out-of-band application security testing (OAST) domains? – e.g. oastify.com, interact.sh, oast.fun?

In [ ]:
%pip install pandas matplotlib

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

In [ ]:
# Enrich search results with associated OAST domains
oast = {
    "project discovery": ["interact.sh", "oast.fun", "interactsh.com", "oast.site", "oast.me", "oast.live", "oast.online", "oast.pro"],
    "tenable": ["nessus.org"],
    "invicti": ["r87.me"],
    "xss hunter": ["xss.ht"],
    "acunetix": ["bxss.me"],
    "bxss hunter": ["bxss.in"],
    "burp": ["oastify.com"],
    "appcheck ng": ["ptst.io"],
    "xss report":["xss.report"],
    "canarytokens":["canarytokens.com"]
}

# If the result contains a signaled value associated to an OAST domain it adds a key called 'oast' with the name of the security tool
# If no matching domain is found, it sets the 'oast' key to "non-oast".
for result in search_results:
    result['oast'] = next((key for key, domains in oast.items() if any(domain in tag['value'] for tag in result['tags'] if tag['type'] == 'OOB-DOMAIN' for domain in domains)), "non-oast")

#### Distribution of OAST domains

In [ ]:
# Create a DataFrame from the search results
df = pd.DataFrame(search_results)
df['oast'].value_counts().nlargest(40).plot(kind='bar', figsize=(10,5))
plt.ylabel('Number of attempts')
plt.xlabel('OAST domain')
plt.show()

#### Breakdown of OAST domains by attack type

In [ ]:

attack_types = ['XSS', 'SQLI', 'XXE', 'CMDEXE', 'LOG4J-JNDI']
filtered_tags = df['tags'].apply(lambda tags: [tag for tag in tags if tag['type'] in attack_types])
filtered_df = pd.DataFrame([(tag['type'], row['oast']) for _, row in df.iterrows() for tag in filtered_tags[_]], columns=['Attack Type', 'OAST'])

filtered_df.groupby(['Attack Type', 'OAST']).size().unstack().plot(kind='bar', figsize=(10, 6))
plt.title('OAST domain by attack type')
plt.xlabel('Attack Type')
plt.ylabel('Count')
plt.show()

### Is the attacker server still live? Can we grab the payload from it?  

In [ ]:
%pip install ipywidgets

In [ ]:
import ipywidgets as widgets

payloads = list({signal['value'] for tags in df['tags'] for signal in tags if signal['type'] == 'OOB-DOMAIN'})

# Create a dropdown widget with extracted payloads
dropdown = widgets.Dropdown(
    options=payloads,
    value=None,
    description='Payloads:',
    disabled=False
)
# # Display the dropdown
display(dropdown)


#### Sandbox scan of OOB Domain using urlscan.io

In [ ]:
# scan oob domain
from pprint import pprint
import re

urlscan_api_key = os.getenv('URLSCAN_API_KEY')

headers = {
    'API-Key': urlscan_api_key,
    'Content-Type':'application/json'
    }

# Extract URL from payload
url_regex = r'https?://[\w\.-]+(?:/[\w\.-]+)*'
urls = list(set(re.findall(url_regex, dropdown.value)))

if not urls:
    print("No URLs found in the selected payload")
    exit()

data = {"url": urls[0], "visibility": "private"}
urlscan_resp = requests.post('https://urlscan.io/api/v1/scan/', headers=headers, data=json.dumps(data))

pprint(urlscan_resp.json())

#### Display screenshot of scanned url

In [ ]:
from PIL import Image
from io import BytesIO
from IPython.display import display

uuid = urlscan_resp.json()['uuid']
headers = {'API-Key': urlscan_api_key}
urlscan_screenshot_resp = requests.post(f'https://urlscan.io/screenshots/{uuid}.png', headers=headers)

# Check if the request was successful
if urlscan_screenshot_resp.status_code == 200:
    # Open the image from the response's content
    image = Image.open(BytesIO(urlscan_screenshot_resp.content))
    # Display the image in the Jupyter Notebook
    display(image)
else:
    print(f"Failed to retrieve the image. Status code: {urlscan_screenshot_resp.status_code}")